<a href="https://colab.research.google.com/github/raf-madrigal/llm-sandbox/blob/main/Multi_Agent_using_Cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install crewai duckduckgo-search cohere serpapi beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install langchain-openai langchain-cohere

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_cohere.chat_models import ChatCohere
from langchain_openai.chat_models import ChatOpenAI

from langchain.agents import load_tools
from google.colab import userdata

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun() #enabling the agents to have access to the internet

os.environ["COHERE_API_KEY"] = userdata.get('COHERE_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_AI_KEY')
os.environ['OPENAI_ORGANIZATION'] = userdata.get('OPEN_AI_ORG')
# Initiate our LLM chat model from Cohere
llm = ChatCohere(model="command-r-plus", temperature=0.3)


In [ ]:
ANALYST_CONFIG = dict(
    role='Senior Data Privacy Officer'
    , goal="Assess the privacy impact of a new AI project aimed at enhancing user experience while maintaining data privacy standards."
    , backstory="""You are a seasoned Senior Data Privacy Officer within a prominent technology firm. Your role is crucial in ensuring that all projects adhere to strict privacy regulations and standards. With a wealth of experience in data protection laws and privacy impact assessments, you are tasked with evaluating the potential privacy implications of new projects and providing recommendations for mitigating risks."""
    , expected_output="Your final deliverable should be a detailed PIA report that serves as a roadmap for addressing privacy concerns and ensuring the responsible use of AI technologies in the project."
)

ANALYST_TASK = """Conduct a comprehensive Privacy Impact Assessment (PIA) for a new AI project focused on enhancing user experience. The project description entails the development of a recommendation system that utilizes machine learning algorithms to personalize content for users based on their browsing history and preferences.

The system aims to improve user engagement and satisfaction while safeguarding user privacy.

Your task involves:

- Analyzing the project description and identifying potential privacy risks associated with the collection, processing, and storage of user data.
- Assessing the legal and regulatory requirements applicable to the project, including GDPR, CCPA, and other relevant data protection laws.
- Evaluating the data handling practices proposed by the project team, including data minimization, purpose limitation, and user consent mechanisms.
- Identifying technical safeguards and security measures to protect against unauthorized access, data breaches, and misuse of personal information.
- Providing recommendations for enhancing privacy protections and compliance with applicable regulations.
- Collaborating with cross-functional teams, including legal, engineering, and product management, to integrate privacy-by-design principles into the project development lifecycle.
- Documenting your findings and recommendations in a comprehensive Privacy Impact Assessment (PIA) report, outlining the potential privacy risks, mitigating measures, and compliance considerations.

Project Name: {project_name}
Project Description: {project_description}"""


ETHICIST_CONFIG = dict(
    role='Senior AI Ethics Officer'
    , goal="Ensure the responsible development and deployment of AI technologies by scrutinizing projects for potential harms and sources of bias, providing fairness considerations, and offering suggestions to mitigate bias."
    , backstory="You are a seasoned Senior AI Ethics Officer with a deep understanding of the ethical implications of artificial intelligence. Having worked in various industries, you've witnessed the transformative power of AI firsthand, but also recognize the importance of mitigating its potential risks. With a background in ethics, technology, and regulatory compliance, you're dedicated to promoting ethical AI practices and safeguarding against unintended consequences."
    , expected_output="Detailed ethics review report that highlights the ethical implications of the AI project, identifies potential harms and sources of bias, and provides actionable recommendations for promoting fairness and responsible AI development."
)

ETHICIST_TASK = """Conduct a comprehensive ethical review of a new AI project aimed at optimizing recommendation algorithms for an e-commerce platform. The project description outlines the use of machine learning models to personalize product recommendations for users based on their browsing and purchasing history. Your task is to scrutinize the project for potential harms and sources of bias, provide fairness considerations, and offer suggestions to mitigate bias.

Your task involves:

Project Assessment: Analyze the project's objectives, methodologies, and data sources to identify potential ethical concerns, such as algorithmic bias, privacy implications, and unintended consequences.
Bias Detection: Examine the training data and algorithms for biases related to race, gender, age, socioeconomic status, or other protected attributes. Utilize fairness metrics and techniques, such as demographic parity and equalized odds, to assess the fairness of the recommendation system.
Impact Assessment: Consider the potential impact of the AI system on diverse user groups, including marginalized communities and vulnerable populations. Evaluate whether the system may perpetuate existing inequalities or reinforce discriminatory practices.
Fairness Considerations: Provide recommendations for enhancing fairness and equity in the recommendation system, such as algorithmic transparency, user control over personalization settings, and diversity-aware training data sampling.
Mitigation Strategies: Propose technical and policy-based solutions to mitigate bias and promote ethical decision-making in the design and implementation of the AI system. This may include algorithmic adjustments, bias-aware training procedures, and robust evaluation frameworks.
Stakeholder Engagement: Collaborate with cross-functional teams, including data scientists, engineers, product managers, and legal experts, to integrate ethical considerations into the project development lifecycle. Engage with external stakeholders, including advocacy groups and affected communities, to solicit feedback and address concerns.
Documentation and Reporting: Document your findings, recommendations, and mitigation strategies in a comprehensive ethics review report. Clearly articulate the potential risks, ethical dilemmas, and proposed solutions to guide decision-making and accountability.

Project Name: {project_name}
Project Description: {project_description}"""

MLR_CONFIG = dict(
    role='Machine Learning Researcher'
    , goal="Synthesize inputs from the Senior AI Ethics Officer and the Senior Data Privacy Officer into a comprehensive report to inform ethical and privacy considerations in AI development."
    , backstory=" You're a Machine Learning Researcher experienced in AI algorithms and ethical frameworks. Your goal is to ensure responsible AI practices, balancing innovation with ethical and privacy concerns."
    , expected_output="A comprehensive report providing actionable guidance for integrating ethical and privacy considerations into the AI project's development, fostering responsible AI practices."
    , task="Review inputs from the Senior AI Ethics Officer and Senior Data Privacy Officer. Integrate ethical frameworks, address biases, and propose privacy-enhancing techniques. Collaborate cross-functionally and produce a concise report outlining recommendations for ethical AI development."
)

def get_analyst_task(project_name, project_description):
  return ANALYST_TASK.format(project_name=project_name, project_description=project_description)

def get_ethicist_task(project_name, project_description):
  return ETHICIST_TASK.format(project_name=project_name, project_description=project_description)

class Agents():

    def __init__(self, project_name, project_description):
        self.project_name = project_name
        self.project_description = project_description
        self.initialize_agents_and_tasks()


    def initialize_agents_and_tasks(self):
        self.agent__analyst = Agent(
                    role=ANALYST_CONFIG['role'],
                    goal=ANALYST_CONFIG['goal'],
                    backstory=ANALYST_CONFIG['backstory'],
                    verbose=True,
                    allow_delegation=False,
                    llm=llm,
                    tools=[search_tool]
                    )

        self.task__analyst = Task(
            description=get_analyst_task(self.project_name, self.project_description),
            expected_output=ANALYST_CONFIG['expected_output'],
            agent=self.agent__analyst,
        )

        self.agent__ethicist = Agent(
                    role=ETHICIST_CONFIG['role'],
                    goal=ETHICIST_CONFIG['goal'],
                    backstory=ETHICIST_CONFIG['backstory'],
                    verbose=True,
                    allow_delegation=False,
                    llm=llm,
                    tools=[search_tool]
                    )

        self.task__ethicist = Task(
            description=get_ethicist_task(self.project_name, self.project_description),
            expected_output=ETHICIST_CONFIG['expected_output'],
            agent=self.agent__ethicist,
        )

        self.agent__mlr = Agent(
                    role=MLR_CONFIG['role'],
                    goal=MLR_CONFIG['goal'],
                    backstory=MLR_CONFIG['backstory'],
                    verbose=True,
                    allow_delegation=False,
                    llm=llm,
                    tools=[search_tool]
                    )

        self.task__mlr = Task(
            description=MLR_CONFIG['task'],
            expected_output=ANALYST_CONFIG['expected_output'],
            agent=self.agent__analyst,
        )

    def get_crew(self):

        crew = Crew(
        agents=[self.agent__analyst, self.agent__ethicist, self.agent__mlr],
        tasks=[self.task__analyst, self.task__ethicist, self.task__mlr],
        verbose=1,
        process=Process.hierarchical,

        manager_llm=ChatCohere(model="command-r-plus", temperature=0.3)
        )

        return crew

PROJECT_DESCRIPTION = """Introducing "DefaultPredict": An AI Model for Predicting Loan Defaults

DefaultPredict is an advanced machine learning model designed specifically to predict the likelihood of a user defaulting on a loan, without consideration for characteristics such as age, gender, or other protected attributes. This model focuses solely on analyzing the historical data related to loan defaults to identify patterns and predictors associated with default tendencies.

Key Features:

Data Integration: DefaultPredict utilizes a comprehensive dataset consisting of historical loan performance data, including factors such as loan amount, repayment terms, credit score, debt-to-income ratio, employment status, and previous defaults. By analyzing these features, the model aims to capture the underlying factors contributing to loan defaults.
Predictive Modeling: Leveraging sophisticated machine learning algorithms, DefaultPredict builds predictive models that learn from past instances of loan defaults. Through iterative training and validation processes, the model identifies significant predictors of default and optimizes its predictive accuracy to generalize well to unseen data.
Feature Engineering: DefaultPredict employs feature engineering techniques to extract relevant information from raw data and enhance the predictive power of the model. This may involve transforming categorical variables into numerical representations, scaling features to a uniform range, or creating interaction terms to capture complex relationships.
Risk Scoring: Based on the predictive models, DefaultPredict assigns a risk score to each loan applicant, indicating the estimated likelihood of default within a specified timeframe (e.g., 12 months). The risk score serves as a quantitative measure of credit risk, enabling lenders to prioritize applications and make informed decisions about loan approvals.
Explainability: DefaultPredict provides interpretability features to explain the factors contributing to each individual's risk score. Through feature importance analysis and model interpretability techniques, the model elucidates the relative importance of different predictors and their impact on default predictions.
Continuous Learning: DefaultPredict is designed for continuous learning and adaptation to evolving trends in loan default behavior. By monitoring model performance over time and incorporating new data, the model remains up-to-date and responsive to changes in economic conditions or lending practices.
Example Usage:

Loan Approval Process: When a user applies for a loan, DefaultPredict analyzes the applicant's profile and generates a risk score indicating the likelihood of default. Lenders use this information to assess credit risk and make data-driven decisions about loan approvals.
Risk Management: Financial institutions utilize DefaultPredict to identify high-risk loan applicants and implement risk mitigation strategies, such as adjusting interest rates, imposing stricter repayment terms, or requiring collateral for higher-risk loans.
Portfolio Monitoring: DefaultPredict helps lenders monitor the credit quality of their loan portfolios and identify potential areas of concern. By regularly updating risk scores and tracking performance metrics, lenders can proactively manage risk and minimize losses associated with defaults.
With DefaultPredict, financial institutions can leverage the power of AI to make more accurate predictions about loan defaults, enabling them to mitigate risk and make informed lending decisions based solely on relevant financial factors.
"""
agents = Agents(project_name='DefaultPredict', project_description=PROJECT_DESCRIPTION)

In [ ]:
WRITER_CONFIG = dict(
    role='Machine Learning Researhcer'
    , backstory="""You are a renowned Tech Content Strategist, known for your insightful and engaging articles on technology and innovation. With a deep understanding of the tech industry, you transform complex concepts into compelling narratives."""
    , goal="Craft compelling content on tech advancements"
    , task="""Using the insights from the researcher’s report, develop an engaging blog post that highlights the most significant AI advancements. , search the internet to validate the real-life use cases.

Report on real-world use cases and provide the organisation name, how they have applied it and the benefits they are experiencing. Your post should be informative yet accessible, catering to a tech-savvy audience.

Aim for a narrative that captures the essence of these breakthroughs and their implications for the future."""
    , expected_output='Final answer MUST be the full blog post of at least 5 paragraphs.'
)


writer = Agent(
  role=WRITER_CONFIG['role'],
  goal=WRITER_CONFIG['goal'],
  backstory=WRITER_CONFIG['backstory'],
  verbose=True,
  allow_delegation=False,
  llm=llm,
  tools=[search_tool]
)

writer_task = Task(
  description=WRITER_CONFIG['task'],
  expected_output=WRITER_CONFIG['expected_output'],
  agent=writer,
#   human_input=True
)

In [ ]:
# crew = Crew(
#     agents=[agent1.agent__analyst, agent1.agent__ethicist],
#     tasks=[agent1.task__analyst, agent1.task__ethicist],
#     verbose=1
# )

crew = agents.get_crew()

result = crew.kickoff()

 [DEBUG]: Working Agent: Crew Manager


> Entering new CrewAgentExecutor chain...
To start, let's evaluate the potential privacy risks associated with the collection, processing, and storage of user data by this project. To do that, I need to understand the types of data being collected, the purpose for its collection, and the methods used to store and process this data. I also need to understand the extent to which user privacy is protected when collecting and processing this data. I will delegate this task to the Senior Data Privacy Officer.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Data Privacy Officer", "task": "Evaluate potential privacy risks", "context": "We're conducting a Privacy Impact Assessment (PIA) for a new AI project focused on enhancing user experience. The project, named DefaultPredict, is an AI Model for Predicting Loan Defaults. It utilizes historical loan performance data, including factors such as loan amount, repayment terms, credit s

In [ ]:
print("######################") # runtime = 12mins ish
print(result)


######################
**Ethics and Privacy Recommendations Report - DefaultPredict Project**

The DefaultPredict project aims to predict the likelihood of a user defaulting on a loan, leveraging machine learning models and historical loan performance data. The project involves the use of sensitive personal data, raising potential privacy concerns and ethical considerations. 

Based on the inputs from the Senior AI Ethics Officer, Senior Data Privacy Officer, and our Machine Learning Researcher, the following comprehensive set of recommendations are proposed:

1. **Conduct a thorough bias audit** of the historical data to ensure it doesn't unfairly disadvantage any groups.

2. **Implement a fairness metric** to measure the model's performance across different groups and adjust the model accordingly for any disparities.

3. **Implement robust data anonymization techniques** and obtain explicit consent from users before using their data.

4. **Engage diverse user groups** in consultation